# Titanic Prediction Data Science (Codsoft)

Importing the necessary dependencies

In [104]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Loading the Dataset from Google Drive

In [105]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Load the Titanic dataset

In [106]:
path = '/content/drive/MyDrive/CodSoft/Titanic-Dataset.csv'
data = pd.read_csv(path)

Exploring the data: \
Display first few rows and summary info


In [107]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [110]:
data.shape

(891, 12)

In [111]:
data.size

10692

Cheking the Statistical informstion of the data

In [112]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Cheking for missing values in dataset

In [113]:
data.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


Droping irrelevant columns

In [114]:
# Droping the irrelevant columns to make the prediction more accurate :-
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [115]:
data.drop(['Fare'] , axis=1 , inplace=True)

Handling missing values in 'Age' using SimpleImputer

In [116]:
imputer = SimpleImputer(strategy='mean')
data[['Age']] = imputer.fit_transform(data[['Age']])

Handling missing values in 'Embarked'

In [117]:
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

Rechecking missing data

In [118]:
print("\nMissing values after handling:")
print(data.isnull().sum())


Missing values after handling:
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64


Rechecking the data

In [119]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.000000,1,0,S
1,1,1,female,38.000000,1,0,C
2,1,3,female,26.000000,0,0,S
3,1,1,female,35.000000,1,0,S
4,0,3,male,35.000000,0,0,S
...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,S
887,1,1,female,19.000000,0,0,S
888,0,3,female,29.699118,1,2,S
889,1,1,male,26.000000,0,0,C


Encoding categorical features (Sex & Embarked)

In [120]:
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['Embarked'] = label_encoder.fit_transform(data['Embarked'])

Splitting the dataset into features X and target Y

In [121]:
X = data.drop(columns=['Survived'])
y = data['Survived']

Splitting the dataset into training and testing data (80% training, 20% testing)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalisation des données avec StandardScaler

In [123]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Modelling

In [124]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,22.000000,1,0,2
1,1,1,0,38.000000,1,0,0
2,1,3,0,26.000000,0,0,2
3,1,1,0,35.000000,1,0,2
4,0,3,1,35.000000,0,0,2
...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,2
887,1,1,0,19.000000,0,0,2
888,0,3,0,29.699118,1,2,2
889,1,1,1,26.000000,0,0,0


In [125]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [126]:
RandomForestClassifier(random_state=42)

RandomForestClassifier(random_state=42)

Make predictions on the test set

In [127]:
y_pred = model.predict(X_test)

Model evaluation

In [128]:
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Model Accuracy: 0.8156

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       105
           1       0.82      0.72      0.76        74

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.81       179
weighted avg       0.82      0.82      0.81       179

Confusion Matrix:
[[93 12]
 [21 53]]


In [129]:
#Cheking the model --:

Testing with new data

In [130]:
new_data = pd.DataFrame({
    'Pclass': [3 , 3],
    'Sex': [1 , 0],  # 0 for female, 1 for male
    'Age': [34.5000 , 47.0000],
    'SibSp': [0  ,1],
    'Parch': [0 , 0],
    'Embarked':[1 , 2]  # 0 for 'C', 1 for 'Q', 2 for 'S'
})

# Scaling new data
new_data_scaled = scaler.transform(new_data)

print(new_data_scaled[:5])

# Predicting survival using the RandomForest model
predictions = model.predict(new_data)

# Display new data predictions
for i, prediction in enumerate(predictions):
    print(f"Passenger {i+1} prediction: {'Survived' if prediction == 1 else 'Not Survived'}")


[[ 0.81303367  0.7243102   0.38210353 -0.47072241 -0.47934164 -0.73076416]
 [ 0.81303367 -1.38062393  1.34472165  0.37992316 -0.47934164  0.5635246 ]]
Passenger 1 prediction: Not Survived
Passenger 2 prediction: Not Survived


In [131]:
# Make predictions using the trained model
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Model Accuracy:", accuracy)

Model Accuracy: 0.8156424581005587


A model accuracy of 0.8156 (or 81.56%) means that the model is making correct predictions for about 81.56% of the instances in the test dataset. In other words, for around 8 out of 10 passengers, the model's survival predictions match the actual survival outcome in the test dataset.
